In [1]:
# Imports
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping

import os, warnings
import os
"""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""

import matplotlib.pyplot as plt
from matplotlib import gridspec

from sklearn.model_selection import train_test_split
from PIL import Image
import PIL.ImageOps  
import glob
import cv2



# Reproductability
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()


# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells

print('complete')


complete


In [3]:
#Resizing to 128,128    
files = os.listdir("../input/aluminum-can-image-data/Aluminum Cans")

for f in files:
    img = Image.open("../input/aluminum-can-image-data/Aluminum Cans/" + f)
    img = img.resize((128,128))


ds_train_ = image_dataset_from_directory(
    '../input/aluminum-can-image-data', 
    labels='inferred',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
)
ds_valid_ = image_dataset_from_directory(
    '../input/aluminum-can-image-data',
    labels='inferred',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
)    

ds_train, ds_valid = train_test_split(files, test_size=0.2, random_state=1)

 
    
# Load training and validation sets    
#ds_train = tf.keras.preprocessing.image_dataset_from_directory(
 # data_dir,
  #labels='inferred',
  #label_mode='binary',
  #validation_split=0.2,
  #subset="training",
  #image_size=(128, 128),
  #batch_size=64)

#ds_valid = tf.keras.preprocessing.image_dataset_from_directory(
 # data_dir,
  #labels='inferred',
  #label_mode='binary',
  #validation_split=0.2,
  #subset="validation",
  #image_size=(128, 128),
  #batch_size=64)
    

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE


ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = ( 
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

Found 133 files belonging to 1 classes.
Found 133 files belonging to 1 classes.


In [4]:
# Early Stopping - Also set callbacks from 'history'
early_stopping = EarlyStopping(
    patience= 10, # how many epochs to wait before stopping
)
print('Early_Stopping Set')

#ReduceLROnPlateau - Reduce learning rate when a metric has stopped improving
ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience =10, 
)
print('ReduceLROnPlateau Set')

Early_Stopping Set
ReduceLROnPlateau Set


In [5]:
model = keras.Sequential([
    layers.InputLayer(input_shape=[128, 128, 3]),
    
    # Data Augmentation
    preprocessing.RandomContrast(factor=0.50),
    preprocessing.RandomFlip(mode='horizontal'),
    preprocessing.RandomRotation(factor=0.50),
    
    # Block One
    layers.BatchNormalization(renorm=True),
    layers.Dropout(rate=0.3),
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block Two
    layers.BatchNormalization(renorm=True),
    layers.Dropout(rate=0.3),
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block Three
    layers.BatchNormalization(renorm=True),
    layers.Dropout(rate=0.3),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Head
    layers.BatchNormalization(renorm=True),
    layers.Flatten(),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])
print('complete')

complete


In [ ]:
optimizer = tf.keras.optimizers.Adam(epsilon=0.01)
model.compile( 
    optimizer=optimizer, 
    loss='mse', 
    metrics=['accuracy'],
) 

history = model.fit( 
    ds_train, 
    validation_data=ds_valid,   
    epochs=150, 
   # callbacks=[early_stopping] 
)

# Plot learning curves
history_frame = pd.DataFrame(history.history) 
history_frame.loc[:, ['loss', 'val_loss']].plot() 
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot(); 

